In [22]:
import math
import wave
import numpy as np

def fft_bins_for_frequency(target_frequency, sample_rate):
    """
    Calculate the number of FFT bins required to detect a signal of a given frequency.

    Parameters:
        target_frequency (float): The frequency of the signal in Hz.
        sample_rate (float): The sampling rate in Hz.

    Returns:
        int: Number of FFT bins required.
    """
    if target_frequency <= 0 or sample_rate <= 0:
        raise ValueError("Target frequency and sample rate must be positive values.")

    # Calculate the minimum time period required to detect the target frequency
    min_time = 1 / target_frequency  # Time period of one cycle of the signal

    # Calculate the minimum number of samples required to observe this time period
    min_samples = int(np.ceil(min_time * sample_rate))

    # The number of FFT bins is equivalent to the number of samples
    fft_bins = min_samples

    return fft_bins

def differentiate_frequencies(freq1, freq2, sample_rate):
    """
    Calculate the minimum number of samples and FFT bins required to differentiate two frequencies.

    Parameters:
        freq1 (float): First frequency in Hz.
        freq2 (float): Second frequency in Hz.
        sample_rate (float): Sampling rate in Hz.

    Returns:
        tuple: (minimum_samples, fft_bins)
    """
    # Ensure frequencies are positive
    freq1, freq2 = abs(freq1), abs(freq2)

    # Calculate the frequency resolution required to differentiate the signals
    delta_freq = abs(freq1 - freq2)
    if delta_freq == 0:
        raise ValueError("The two frequencies must be different to differentiate them.")

    # Calculate the minimum observation time (T) needed to distinguish the frequencies
    min_time = 1 / delta_freq

    # Minimum number of samples is the observation time multiplied by the sampling rate
    min_samples = int(np.ceil(sample_rate * min_time))

    return min_samples

def generate_sine_wave(frequency, other_frequency, sample_rate):
    """
    Generate a single full cycle of a sine wave at a given frequency.

    Args:
        frequency (float): Frequency of the sine wave in Hz.
        sample_rate (int): Sampling rate in Hz (default is 44100 Hz).

    Returns:
        numpy.ndarray: Array containing the samples of the sine wave.
    """
    # Calculate the number of samples for one cycle
    samples_per_cycle = int(sample_rate / frequency)

    # Calculate the number of samples needed for the FFT
    if frequency != other_frequency:
        samples_to_differentiate = differentiate_frequencies(frequency, other_frequency, sample_rate)
    else:
        samples_to_differentiate = fft_bins_for_frequency(frequency, sample_rate)

    samples_to_transmit = math.ceil(samples_to_differentiate / float(samples_per_cycle)) * samples_per_cycle

    # Generate time values for one full cycle
    t = np.linspace(0, 1 / frequency, samples_to_transmit, endpoint=False)

    # Generate the sine wave
    sine_wave = np.sin(2 * np.pi * frequency * t)
    return sine_wave

def save_wave_file(filename, wave_data, sample_rate=44100):
    """
    Save a numpy array as a wave file.

    Args:
        filename (str): Output filename.
        wave_data (numpy.ndarray): Waveform data to save.
        sample_rate (int): Sampling rate in Hz (default is 44100 Hz).
    """

    # Write to a wave file
    with wave.open(filename, 'w') as wav_file:
        wav_file.setnchannels(1)  # Mono
        wav_file.setsampwidth(2)  # 16-bit
        wav_file.setframerate(sample_rate)
        for data in wave_data:
            # Convert waveform data to 16-bit PCM format
            converted = (data * 32767).astype(np.int16)
            wav_file.writeframes(converted.tobytes())

def generate_random_bits(length):
    """
    Generate an array of random bits (0 or 1).

    Args:
        length (int): The length of the array.

    Returns:
        numpy.ndarray: Array of random bits.
    """
    return np.random.randint(0, 2, length, dtype=np.uint8)

# Example usage
base = 440.0  # Frequency of the sine wave in Hz (e.g., A4)
tone = 1.0
copies = 10
sample_rate = 44100
output_file = 'sine_wave_cycle.wav'

messageSize = 256 * 8 # in bits
message = generate_random_bits(messageSize)

results = [generate_sine_wave(base * tone, base * tone, sample_rate)]

for bit in message:
    if bit == 1:
        tone = tone * 3.0
        other = tone * 5.0
    else:
        tone = tone * 5.0
        other = tone * 3.0

    while tone > 2.0:
        tone = tone / 2.0

    while other > 2.0:
        other = other / 2.0

    # Generate the sine wave
    for i in range(copies):
        sine_wave = generate_sine_wave(frequency * tone, frequency * other, sample_rate)
        results = results + [sine_wave]

# Save the sine wave as a wave file
save_wave_file(output_file, results)
print(f"Generated audio file: {output_file}")


Generated audio file: sine_wave_cycle.wav
